In [21]:
import sys; sys.path.append('..'); sys.path.append('../..')
from common.postgresql import PostgresConnector
import pandas as pd
import re

db = PostgresConnector()
read_sql = db.read_sql
sql = db.execute_sql

In [45]:

TABLES_MAP = {
    'fi_cad_fi_hist_admin': ['cnpj_admin', 'admin', 'dt_ini_admin', 'dt_fim_admin'],
    'fi_cad_fi_hist_auditor': ['cnpj_auditor', 'auditor', 'dt_fim_auditor'],
    'fi_cad_fi_hist_classe': ['classe', 'dt_fim_classe'],
    'fi_cad_fi_hist_condom': ['condom', 'dt_fim_condom'],
    'fi_cad_fi_hist_controlador': ['cnpj_controlador', 'controlador', 'dt_fim_controlador'],
    'fi_cad_fi_hist_custodiante': ['cnpj_custodiante', 'custodiante', 'dt_fim_custodiante'],
    'fi_cad_fi_hist_denom_social': ['denom_social', 'dt_fim_denom_social'],
    'fi_cad_fi_hist_denom_comerc': ['denom_comerc', 'dt_fim_denom_comerc'],
    'fi_cad_fi_hist_diretor_resp': ['diretor', 'dt_fim_diretor'],
    'fi_cad_fi_hist_exclusivo': ['fundo_exclusivo', 'dt_fim_st_exclusivo'],
    'fi_cad_fi_hist_fic': ['fundo_cotas', 'dt_fim_st_cotas'],
    'fi_cad_fi_hist_gestor': ['pf_pj_gestor', 'cpf_cnpj_gestor', 'gestor', 'dt_ini_gestor', 'dt_fim_gestor'],
    'fi_cad_fi_hist_publico_alvo': ['publico_alvo', 'dt_fim_publico_alvo'],
    'fi_cad_fi_hist_rentab': ['rentab_fundo', 'dt_fim_rentab'],
    'fi_cad_fi_hist_sit': ['sit', 'dt_ini_sit', 'dt_fim_sit'],
    'fi_cad_fi_hist_taxa_adm': ['taxa_adm', 'inf_taxa_adm'],
    'fi_cad_fi_hist_taxa_perfm': ['vl_taxa_perfm', 'ds_taxa_perfm'],
    'fi_cad_fi_hist_trib_lprazo': ['trib_lprazo', 'dt_fim_st_trib_lprazo']
}

In [46]:
from copy import copy
from abc import ABC
# class Tables(ABC): pass
# _tables = Tables()
_tables = {}
for table_name in TABLES_MAP.keys():
    _tables[table_name] = read_sql(f"SELECT * FROM cvm.{table_name}")
    # setattr(_tables, table_name, read_sql(f"SELECT * FROM cvm.{table_name}"))

In [47]:
final_cols = [
    'cnpj_fundo', 'dt_reg', 'dt_ini', 
    'cnpj_admin', 'admin', 
    'cnpj_auditor', 'auditor',
    'cnpj_custodiante', 'custodiante',
    'classe', 'condom', 
    'cnpj_controlador', 'controlador',
    'denom_comerc', 'denom_social', 'diretor',
    'fundo_exclusivo', 'fundo_cotas',
    'pf_pj_gestor', 'cpf_cnpj_gestor', 'gestor',
    'publico_alvo', 'rentab_fundo', 'sit',
    'taxa_adm', 'inf_taxa_adm', 'vl_taxa_perfm', 'ds_taxa_perfm', 'trib_lprazo',
    'dt_fim'
]

In [48]:
from copy import copy
tables = copy(_tables)

In [ ]:
for table_name, df in tables.items():
    rename_dict = {}
    
    ini_matches = [c for c in df.columns if 'dt_ini' in c]
    if ini_matches:
        rename_dict[ini_matches[0]] = 'dt_ini'
        
    fim_matches = [c for c in df.columns if 'dt_fim' in c]
    if fim_matches:
        rename_dict[fim_matches[0]] = 'dt_fim'
        
    df.rename(columns=rename_dict, inplace=True)
    df = df[list(set(df.columns).intersection(final_cols))]
    if table_name in ['fi_cad_fi_hist_taxa_perfm', 'fi_cad_fi_hist_taxa_adm']:
        df.loc[:, 'dt_fim'] = None
    tables[table_name] = df

In [42]:
# for key in tables.keys():
#     cols = ['cnpj_fundo', 'dt_reg', 'dt_ini', 'dt_fim']
#     print(key, set(tables[key].columns).intersection(cols))

In [50]:
from functools import reduce
import pandas as pd

df_merged = reduce(lambda left, right: pd.merge(left, right, on=["cnpj_fundo", "dt_reg", "dt_ini", "dt_fim"], how="outer"), tables.values())


In [41]:
df_merged.columns

Index(['cnpj_fundo', 'dt_ini', 'cnpj_admin', 'admin', 'dt_fim', 'dt_reg',
       'auditor', 'cnpj_auditor', 'classe', 'condom', 'controlador',
       'cnpj_controlador', 'custodiante', 'cnpj_custodiante', 'denom_comerc',
       'diretor', 'fundo_exclusivo', 'fundo_cotas', 'cpf_cnpj_gestor',
       'gestor', 'pf_pj_gestor', 'publico_alvo', 'rentab_fundo', 'sit',
       'taxa_adm', 'inf_taxa_adm', 'vl_taxa_perfm', 'ds_taxa_perfm',
       'trib_lprazo'],
      dtype='object')

In [52]:
df_merged[(df_merged['denom_social'].str.upper().str.contains('NIX'))&(df_merged['gestor'].str.upper().str.contains('KINEA'))][['denom_social', 'cnpj_fundo' ,'gestor', 'sit', 'dt_fim']]

,denom_social,cnpj_fundo,gestor,sit,dt_fim
744663,KINEA NIX FUNDO DE INVESTIMENTO FINANCEIRO MUL...,58.953.805/0001-30,KINEA INVESTIMENTOS LTDA.,NaN,None


In [ ]:
tables['fi_cad_fi_hist_sit'][tables['fi_cad_fi_hist_sit']['cnpj_fundo'] == '58.953.805/0001-30']

,cnpj_fundo,dt_ini,sit,dt_fim,dt_reg
133266,58.953.805/0001-30,2025-01-17,FASE PRÉ-OPERACIONAL,2025-01-31,2025-01-17
133267,58.953.805/0001-30,2025-01-31,EM FUNCIONAMENTO NORMAL,None,2025-01-17


In [ ]:
df = tables.fi_cad_fi_hist_admin

,cnpj_fundo,cnpj_admin,dt_fim,admin,dt_ini,dt_reg
0,00.000.432/0001-00,33.868.597/0001-40,2005-04-01,CITIBANK DTVM SA,2005-03-31,2005-03-31
1,00.000.432/0001-00,61.472.676/0001-72,2006-11-21,BANCO SANTANDER BRASIL SA,2005-04-01,2005-03-31
2,00.000.432/0001-00,90.400.888/0001-42,2008-07-18,BANCO SANTANDER (BRASIL) S.A.,2006-11-21,2005-03-31
3,00.000.744/0001-06,33.066.408/0001-15,2006-06-30,BANCO ABN AMRO REAL S.A.,2005-01-01,2005-01-13
4,00.000.746/0001-03,60.770.336/0001-65,None,BANCO ALFA DE INVESTIMENTO S.A.,2004-11-29,2004-12-09
...,...,...,...,...,...,...
82247,97.929.197/0001-80,02.201.501/0001-61,2021-06-15,BNY MELLON SERVICOS FINANCEIROS DTVM S.A.,2013-11-25,2011-08-31
82248,97.929.197/0001-80,59.281.253/0001-23,None,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,2021-06-15,2011-08-31
82249,97.929.213/0001-34,61.809.182/0001-30,2022-12-16,UBS (BRASIL) CORRETORA DE VALORES S.A,2011-08-07,2011-08-31
82250,97.929.213/0001-34,02.332.886/0001-04,2025-12-15,XP INVESTIMENTOS CCTVM S.A.,2022-12-16,2011-08-31
